In [2]:
import cx_Oracle
import pandas as pd
import os
import string
from dfply import *

def file_to_string(rel_file_path):
    with open(rel_file_path, 'r') as ftoread:
        fstring = ftoread.read()
    return fstring

#date_start = "01-JAN-2014"
date_start = "01-OCT-2016"
date_end = "15-JUN-2017"
db = cx_Oracle.connect(os.environ.get('CONN_ORACLEWPL'))

In [3]:
t = string.Template(file_to_string('../jzn1.sql')).substitute(ds=date_start, de=date_end)
print(t)

-- Grab total coliform & total chlorine in sites that are not part of the total coliform moni
-- SAMPLE_DATE, SOURCE, SAMPLE_ID, L_ANALYTE, L_VALUE, F_ANALYTE, F_VALUE, F_FIELD_RECORD
SELECT DISTINCT
SAMPLE_DATE as SAMPLE_DATE, 
s.SOURCE as SOURCE, 
s.sample_id as SAMPLE_ID, 
r.analyte as L_ANALYTE, 
r.qualifier as L_VALUE, 
f.analyte as F_ANALYTE, 
f.qualifier||f.value as F_VALUE,  
f.field_record as F_FIELD_RECORD
FROM
WPL.SAMPLE s, WPL.FIELD_DATA f, WPL.RESULT r, WPL.XR_SOURCE xrs
WHERE s.sample_id=r.sample_id
AND test_number=1
AND test_type='SAMP'
AND r.analyte in ('T_COLIFORM')
AND r.reportable=1
AND f.analyte in ('CL2_RES_TOTAL')
AND s.Source in (SELECT DISTINCT SOURCE FROM WPL.XR_SOURCE WHERE SOURCE_NAME = 'SYS' AND VALID='Y')
and S.SOURCE not in ('119 SYS','136 SYS','15A SYS','340 SYS','344 SYS')
AND SAMPLE_DATE BETWEEN '01-OCT-2016' AND '15-JUN-2017'
AND s.sample_id=r.sample_id
AND s.field_record=f.field_record AND xrs.source=s.source


In [ ]:
jzn_2_q = string.Template("""
-- Select 2nd part of analysis - ecoli linke with view 1
-- (SAMPLE_DATE, SOURCE, SAMPLE_ID, L_ANALYTE, L_VALUE, F_ANALYTE, F_VALUE, F_FIELD_RECORD)
SELECT DISTINCT
s.SAMPLE_DATE as SAMPLE_DATE, 
s.SOURCE as SOURCE, 
s.sample_id as SAMPLE_ID, 
r.analyte as L_ANALYTE, 
r.qualifier as L_VALUE, 
f.analyte as F_ANALYTE,
' '||f.qualifier||' '||f.value AS F_VALUE,
f.field_record as F_FIELD_RECORD
FROM
WPL.SAMPLE s, WPL.FIELD_DATA f, WPL.RESULT r, WPL.XR_SOURCE xrs
WHERE s.sample_id=r.sample_id
AND test_number=1
AND test_type='SAMP'
AND r.analyte in ('E_COLI')
AND r.reportable=1
AND f.analyte in ('CL2_RES_TOTAL')
AND s.Source in (SELECT DISTINCT SOURCE FROM WPL.XR_SOURCE WHERE SOURCE_NAME = 'SYS' AND VALID='Y')
AND SAMPLE_DATE BETWEEN '$ds' AND '$de'
AND s.sample_id=r.sample_id
AND s.field_record=f.field_record
AND xrs.source=s.source
""").substitute(ds=date_start, de=date_end)

In [ ]:
jzn_3_q = string.Template("""
-- Use total coliform again and link to field data of temp
-- (F_FIELD_RECORD, F_ANALYTE, F_QUAL, F_VALUE)
SELECT DISTINCT f.field_record as F_FIELD_RECORD, 
f.analyte as F_ANALYTE, 
f.qualifier as F_QUAL,
f.value as F_VALUE
FROM
WPL.SAMPLE s, WPL.FIELD_DATA f, WPL.RESULT r
WHERE r.analyte in ('T_COLIFORM')
AND f.analyte in ('TEMPERATURE')
AND s.source in (SELECT DISTINCT SOURCE FROM WPL.XR_SOURCE WHERE SOURCE_NAME ='SYS' AND VALID='Y')
AND SAMPLE_DATE BETWEEN '$ds' AND '$de'
AND s.sample_id=r.sample_id
AND s.field_record=f.field_record
""").substitute(ds=date_start, de=date_end)

In [ ]:
jzn_4_q = string.Template("""
-- Ecoli linked with field record ph
-- (F_FIELD_RECORD, F_ANALYTE, F_QUAL, F_VALUE)
SELECT DISTINCT
f.field_record as F_FIELD_RECORD, 
f.analyte as F_ANALYTE, 
f.qualifier as F_QUAL, 
f.value as F_VALUE
FROM
WPL.SAMPLE s, WPL.FIELD_DATA f, WPL.RESULT r
WHERE r.analyte in ('E_COLI')
AND f.analyte in ('PH')
AND s.source in (SELECT DISTINCT SOURCE FROM WPL.XR_SOURCE WHERE SOURCE_NAME ='SYS' AND VALID='Y')
AND SAMPLE_DATE BETWEEN '$ds' AND '$de'
AND s.sample_id=r.sample_id
AND s.field_record=f.field_record
""").substitute(ds=date_start, de=date_end)

In [ ]:
jzn_1 = pd.read_sql_query(jzn_1_q, db, coerce_float=True, index_col='F_FIELD_RECORD')
jzn_1.F_VALUE = pd.to_numeric(jzn_1.F_VALUE, errors='coerce')
jzn_1 = jzn_1[jzn_1.F_VALUE.notnull()]
jzn_1.head()


In [ ]:
jzn_2 = pd.read_sql_query(jzn_2_q, db, coerce_float=True, index_col='F_FIELD_RECORD')
jzn_2.F_VALUE = pd.to_numeric(jzn_2.F_VALUE, errors='coerce')
jzn_2 = jzn_2[jzn_2.F_VALUE.notnull()]
jzn_2.head()

In [ ]:
jzn_3 = pd.read_sql_query(jzn_3_q, db, coerce_float=True, index_col='F_FIELD_RECORD')
jzn_3.F_VALUE = pd.to_numeric(jzn_3.F_VALUE, errors='coerce')
jzn_3 = jzn_3[jzn_3.F_VALUE.notnull()]
jzn_3.head()

In [ ]:
jzn_4 = pd.read_sql_query(jzn_4_q, db, coerce_float=True, index_col='F_FIELD_RECORD')
jzn_4.F_VALUE = pd.to_numeric(jzn_4.F_VALUE, errors='coerce')
jzn_4 = jzn_4[jzn_4.F_VALUE.notnull()]
jzn_4.tail()

In [ ]:
jn_1 = jzn_1.rename(columns = {
    'SOURCE':'V5_SOURCE',
    'SAMPLE_DATE':'V5_SAMPLE_DATE',
    'SAMPLE_ID':'V5_SAMPLE_ID',
    'F_VALUE':'V5_CL2_TOTAL',
    'L_VALUE':'V5_T_COLIFORM'
})
jn_1 = jn_1.filter(like='V5',axis=1)
jn_1.head()

In [ ]:
jn_2 = jzn_2.rename(columns = {
    'L_VALUE':'V5_E_COLI'
})
jn_2 = jn_2.filter(like='V5',axis=1)
jn_2.head()

In [ ]:
jn_3 = jzn_3.rename(columns = {
    'F_QUAL':'V5_TEMP_PART1',
    'F_VALUE':'V5_TEMP_PART2'
})
jn_3 = jn_3.filter(like='V5',axis=1)
jn_3.head()

In [ ]:
jn_4 = jzn_4.rename(columns = {
    'F_QUAL':'V5_PH_PART1',
    'F_VALUE':'V5_PH_PART2'
}).filter(like='V5',axis=1)
jn_4.head()

In [ ]:
df = jn_1.join([jn_2, jn_3, jn_4], how='inner')
df.head()

In [ ]:
df = df.rename(columns = {
    'V5_PH_PART2':'V5_PH',
    'V5_TEMP_PART2':'V5_TEMPERATURE'
})

del df['V5_PH_PART1']
del df['V5_TEMP_PART1']

df.head()

In [ ]:
df = df.rename(columns=lambda x: re.sub('V5\_','',x))
df.index.rename(name='FR_NUM', inplace=True)
df.head()

In [ ]:
df.index.rename(name='FR_NUM', inplace=True)
df.head()

In [ ]:
df.to_csv('./test.csv', index=True, encoding='utf-8', doublequote=True, date_format="%Y-%m-%d %H:%M:%S")